# Adult data analysis

In this notebook we do some basic analysis of the adult data, to understand existing biases before training a model.

In [ ]:
from pathlib import Path

import pandas as pd
import plotly.graph_objs as go
from helpers.finance import bin_hours_per_week

In [ ]:
from helpers import export_plot

Directory for saving all processed data.

In [ ]:
artifacts_dir = Path("../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../artifacts")

Load the preprocessed data. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
data_dir = artifacts_dir / "data" / "adult"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

train_oh = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val_oh = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test_oh = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

## Bias in data

Visualise biases present in the data

In [ ]:
salary_by_sex = train[["sex", "salary"]].groupby("sex").mean()

fig_salary_by_sex = go.Figure(
    [go.Bar(x=["Female", "Male"], y=salary_by_sex.salary)],
    go.Layout(yaxis={"range": [0, 1]}),
)
fig_salary_by_sex

In [ ]:
export_plot(fig_salary_by_sex, "salary_by_sex.json")

In [ ]:
salary_by_race = train[["race", "salary"]].groupby("race").mean()

fig_salary_by_race = go.Figure(
    [
        go.Bar(
            x=[
                "American Indian / Eskimo",
                "Asian / Pacific Islander",
                "Black",
                "Other",
                "White",
            ],
            y=salary_by_race.salary,
        )
    ],
    go.Layout(yaxis={"range": [0, 1]}),
)
fig_salary_by_race

In [ ]:
export_plot(fig_salary_by_race, "salary_by_race.json")

### Possible resolving variables

Let's look at the relationship between hours per week and salary.

In [ ]:
salary_by_hours_per_week = (
    val.assign(hpw=val.hours_per_week.map(bin_hours_per_week))
    .loc[:, ["hpw", "salary"]]
    .groupby("hpw")
    .aggregate(["mean", "count"])
    .reset_index()
)

fig_salary_by_hours_per_week = go.Figure(
    data=[
        go.Bar(
            x=salary_by_hours_per_week.hpw,
            y=salary_by_hours_per_week["salary"]["mean"],
        )
    ],
    layout={
        "title": "Proportion of salary > $50k by hours worked per week",
        "yaxis": {"range": [0, 1], "title": "Proportion salary > $50k"},
        "xaxis": {
            "tickvals": [0, 1, 2, 3],
            "ticktext": ["<30hrs", "30-40hrs", "40-50hrs", ">50hrs"],
        },
    },
)
fig_salary_by_hours_per_week

In [ ]:
export_plot(
    fig_salary_by_hours_per_week, "salary_by_hours_per_week.json",
)